In [25]:
import pandas as pd
from IPython.display import display
from pandas.tseries.holiday import USFederalHolidayCalendar

In [2]:
wego = pd.read_csv("../data/Route 50 Timepoint and Headway Data, 1-1-2023 through 5-12-2025.csv")

# WeGo Public Transit
[WeGo Public Transit](https://www.wegotransit.com/) is a public transit system serving the Greater Nashville and Davidson County area. WeGo provides local and regional bus routes, the WeGo Star train service connecting Lebanon to downtown Nashville, along with several other transit services.

The data for this project can be downloaded from [here](https://drive.google.com/drive/folders/1L8d3xEaPD13BMz_k-3G8XRRLvPIbNRq9?usp=sharing).

Since 2019, WeGo has been using [**Transit Signal Priority (TSP)**](https://www.wegotransit.com/projects/transit-signal-priority/), a technology that helps to manage traffic flow more efficiently. For buses it reduces wait times at traffic signals by holding green lights longer, shortening red lights or in some cases allowing buses to bypass traffic. 

The data that you have been provided was collected for trips on Route 50, Charlotte Pike. TSP has been used on portions of this route, with different periods of being on or off, either conditionally or unconditionally. For these timespans, TSP was used between White Bridge and MCC, including all intervening timepoints, in both directions.
The important dates are as follows:

* February 3rd @ 12 noon: TSP Turned On (Unconditional)

* February 10th @ 12 noon: TSP Schedule-Conditional Priority Begins (Only buses more than 2 minutes late receive priority)

* April 28th @ 12 noon: TSP Turned Off

* May 5th @ 12 noon: TSP Turned On (Unconditional)

* May 12th @ 12 noon: TSP Headway-Conditional TSP Priority Begins (Only gapped buses with actual leading headway more than 120% of scheduled headway receive priority)


In [3]:
wego.head()

,CALENDAR_ID,SERVICE_ABBR,ADHERENCE_ID,DATE,ROUTE_ABBR,BLOCK_ABBR,OPERATOR,TRIP_ID,OVERLOAD_ID,ROUTE_DIRECTION_NAME,...,ACTUAL_HDWY,HDWY_DEV,ADJUSTED_EARLY_COUNT,ADJUSTED_LATE_COUNT,ADJUSTED_ONTIME_COUNT,STOP_CANCELLED,PREV_SCHED_STOP_CANCELLED,IS_RELIEF,BLOCK_STOP_ORDER,DWELL_IN_MINS
0,120230101,3,93549161,2023-01-01,50,5000,2355,332422,0,TO DOWNTOWN,...,NaN,NaN,0,0,1,0,0.0,0,2,8.133333
1,120230101,3,93549162,2023-01-01,50,5000,2355,332422,0,TO DOWNTOWN,...,NaN,NaN,0,0,1,0,0.0,0,5,0.000000
2,120230101,3,93549163,2023-01-01,50,5000,2355,332422,0,TO DOWNTOWN,...,NaN,NaN,0,0,1,0,0.0,0,11,0.000000
3,120230101,3,93549164,2023-01-01,50,5000,2355,332422,0,TO DOWNTOWN,...,NaN,NaN,0,0,1,0,0.0,0,13,0.000000
4,120230101,3,93549165,2023-01-01,50,5000,2355,332422,0,TO DOWNTOWN,...,NaN,NaN,0,0,1,0,0.0,0,18,2.150000


In [27]:
(
    wego
    .loc[wego['CALENDAR_ID'] == 120240203]
    .loc[wego['TRIP_ID'] == 371878]
    [[
        'DATE', 'CALENDAR_ID', 'TRIP_ID', 'ROUTE_ABBR',
        'TIME_POINT_ABBR', 'TRIP_EDGE',
        'SCHEDULED_TIME', 'ACTUAL_DEPARTURE_TIME', 'ADHERENCE',
        'ADJUSTED_EARLY_COUNT', 'ADJUSTED_LATE_COUNT', 'ADJUSTED_ONTIME_COUNT'
    ]]
)

,DATE,CALENDAR_ID,TRIP_ID,ROUTE_ABBR,TIME_POINT_ABBR,TRIP_EDGE,SCHEDULED_TIME,ACTUAL_DEPARTURE_TIME,ADHERENCE,ADJUSTED_EARLY_COUNT,ADJUSTED_LATE_COUNT,ADJUSTED_ONTIME_COUNT
282333,2024-02-03,120240203,371878,50,WALM,1,05:34:00,05:34:45,-0.750000,0,0,1
282334,2024-02-03,120240203,371878,50,HLWD,0,05:40:00,05:40:04,-0.066666,0,0,1
282335,2024-02-03,120240203,371878,50,WHBG,0,05:47:00,05:45:43,1.283333,1,0,0
282336,2024-02-03,120240203,371878,50,CH46,0,05:50:00,05:50:40,-0.666666,0,0,1
282337,2024-02-03,120240203,371878,50,28&CHARL,0,05:54:00,05:54:52,-0.866666,0,0,1
282338,2024-02-03,120240203,371878,50,MCC5_1,2,06:05:00,06:04:18,0.700000,0,0,1


In [5]:
wego['DAY_TYPE'] = wego['SERVICE_ABBR'].replace({1: 'Weekday', 2: 'Saturday', 3: 'Sunday'}).astype('category')

In [46]:
holiday_cal = USFederalHolidayCalendar()
holidays = holiday_cal.holidays(start=wego['DATE'].min(), end=wego['DATE'].max())

In [47]:
holidays

DatetimeIndex(['2023-01-02', '2023-01-16', '2023-02-20', '2023-05-29',
               '2023-06-19', '2023-07-04', '2023-09-04', '2023-10-09',
               '2023-11-10', '2023-11-23', '2023-12-25', '2024-01-01',
               '2024-01-15', '2024-02-19', '2024-05-27', '2024-06-19',
               '2024-07-04', '2024-09-02', '2024-10-14', '2024-11-11',
               '2024-11-28', '2024-12-25', '2025-01-01', '2025-01-20',
               '2025-02-17'],
              dtype='datetime64[ns]', freq=None)

In [48]:
wego['IS_HOLIDAY'] = wego['DATE'].isin(holidays)
wego

,CALENDAR_ID,SERVICE_ABBR,ADHERENCE_ID,DATE,ROUTE_ABBR,BLOCK_ABBR,OPERATOR,TRIP_ID,OVERLOAD_ID,ROUTE_DIRECTION_NAME,...,ADJUSTED_EARLY_COUNT,ADJUSTED_LATE_COUNT,ADJUSTED_ONTIME_COUNT,STOP_CANCELLED,PREV_SCHED_STOP_CANCELLED,IS_RELIEF,BLOCK_STOP_ORDER,DWELL_IN_MINS,DAY_TYPE,IS_HOLIDAY
0,120230101,3,93549161,2023-01-01,50,5000,2355,332422,0,TO DOWNTOWN,...,0,0,1,0,0.0,0,2,8.133333,Sunday,False
1,120230101,3,93549162,2023-01-01,50,5000,2355,332422,0,TO DOWNTOWN,...,0,0,1,0,0.0,0,5,0.000000,Sunday,False
2,120230101,3,93549163,2023-01-01,50,5000,2355,332422,0,TO DOWNTOWN,...,0,0,1,0,0.0,0,11,0.000000,Sunday,False
3,120230101,3,93549164,2023-01-01,50,5000,2355,332422,0,TO DOWNTOWN,...,0,0,1,0,0.0,0,13,0.000000,Sunday,False
4,120230101,3,93549165,2023-01-01,50,5000,2355,332422,0,TO DOWNTOWN,...,0,0,1,0,0.0,0,18,2.150000,Sunday,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
618993,120250512,1,119531049,2025-05-12,50,8401,3077,432121,0,FROM DOWNTOWN,...,1,0,0,0,NaN,0,23,4.500000,Weekday,False
618994,120250512,1,119531638,2025-05-12,50,9302,3246,432353,0,TO DOWNTOWN,...,0,0,1,0,0.0,0,2,27.283333,Weekday,False
618995,120250512,1,119531639,2025-05-12,50,9302,3246,432353,0,TO DOWNTOWN,...,0,0,1,0,NaN,0,3,0.133333,Weekday,False
618996,120250512,1,119531770,2025-05-12,50,9950,2448,432387,0,TO DOWNTOWN,...,0,0,1,0,0.0,0,2,8.166666,Weekday,False


IS_HOLIDAY
False    618998
Name: count, dtype: int64


The first main variable you will be studying in this project is **adherence**, which compares the actual departure time to the scheduled time and is included in the ADHERENCE column. A negative adherence value means that a bus left a time point late and a positive adherence indicates that the bus left the time point early. Buses with adherence values beyond negative 6 are generally considered late and beyond positive 1 are considered early. However, there is some additional logic where the staff applies waivers to allow early departures. For example, express buses that have already picked up everyone at a park-and-ride lot and are only dropping off passengers may be allowed to leave early.  Early departures are also permitted at the end of a trip (when TRIP_EDGE = 2), since they do not affect upstream passengers. **Note:** When determining whether a bus is early or late, it is advised that you use the 'ADJUSTED_EARLY_COUNT', 'ADJUSTED_LATE_COUNT', and 'ADJUSTED_ONTIME_COUNT' columns in order to account for the adjustments.

In [36]:
adjusted_counts = wego[['DATE', 'DAY_TYPE', 'CALENDAR_ID', 'IS_HOLIDAY', 'OPERATOR', 'TRIP_ID', 'TIME_POINT_ABBR', 'TRIP_EDGE', 'SCHEDULED_TIME', 'ACTUAL_ARRIVAL_TIME', 'ADHERENCE', 'DWELL_IN_MINS', 'SCHEDULED_HDWY', 'ACTUAL_HDWY', 'HDWY_DEV', 'ROUTE_DIRECTION_NAME', 'ADJUSTED_EARLY_COUNT', 'ADJUSTED_LATE_COUNT', 'ADJUSTED_ONTIME_COUNT']]

In [7]:
#adjusted_counts

In [8]:
# adjusted_counts.loc[(adjusted_counts['CALENDAR_ID'] == 120250203) & (adjusted_counts['ADHERENCE'] < -6)]

In [9]:
# adjusted_counts.loc[(adjusted_counts['CALENDAR_ID'] == 120250210) & (adjusted_counts['ADHERENCE'] < -6)]

In [10]:
# adjusted_counts.loc[(adjusted_counts['CALENDAR_ID'] == 120250428) & (adjusted_counts['ADHERENCE'] < -6)]

In [11]:
# adjusted_counts.loc[(adjusted_counts['CALENDAR_ID'] == 120250505) & (adjusted_counts['ADHERENCE'] < -6)]

In [37]:
adjusted_counts.loc[
    (adjusted_counts['CALENDAR_ID'] == 120250512) & 
    (adjusted_counts['ADHERENCE'] < -6) & 
    (adjusted_counts['TIME_POINT_ABBR'] == 'WHBG')]

,DATE,DAY_TYPE,CALENDAR_ID,IS_HOLIDAY,OPERATOR,TRIP_ID,TIME_POINT_ABBR,TRIP_EDGE,SCHEDULED_TIME,ACTUAL_ARRIVAL_TIME,ADHERENCE,DWELL_IN_MINS,SCHEDULED_HDWY,ACTUAL_HDWY,HDWY_DEV,ROUTE_DIRECTION_NAME,ADJUSTED_EARLY_COUNT,ADJUSTED_LATE_COUNT,ADJUSTED_ONTIME_COUNT
618450,2025-05-12,Weekday,120250512,False,2109,429504,WHBG,0,10:16:00,10:25:23,-9.383333,0.000000,15.0,22.933333,7.933333,TO DOWNTOWN,0,1,0
618457,2025-05-12,Weekday,120250512,False,2109,429505,WHBG,0,11:03:00,11:09:50,-6.833333,0.000000,15.0,20.633333,5.633333,FROM DOWNTOWN,0,1,0
618774,2025-05-12,Weekday,120250512,False,1587,429628,WHBG,0,12:46:00,12:50:21,-6.533333,2.183333,15.0,20.216666,5.216666,TO DOWNTOWN,0,1,0
618793,2025-05-12,Weekday,120250512,False,1764,429631,WHBG,0,15:07:00,15:13:20,-6.333333,0.000000,15.0,20.200000,5.200000,FROM DOWNTOWN,0,1,0
618810,2025-05-12,Weekday,120250512,False,2362,429634,WHBG,0,17:46:00,17:56:24,-12.983333,2.583333,17.0,26.300000,9.300000,TO DOWNTOWN,0,1,0
618817,2025-05-12,Weekday,120250512,False,2362,429635,WHBG,0,18:33:00,18:39:07,-6.116666,0.000000,15.0,19.550000,4.550000,FROM DOWNTOWN,0,1,0
618889,2025-05-12,Weekday,120250512,False,3328,429681,WHBG,0,09:18:00,09:25:51,-7.850000,0.000000,15.0,22.183333,7.183333,FROM DOWNTOWN,0,1,0
618937,2025-05-12,Weekday,120250512,False,2706,429689,WHBG,0,15:22:00,15:30:01,-8.016666,0.000000,15.0,16.683333,1.683333,FROM DOWNTOWN,0,1,0
618942,2025-05-12,Weekday,120250512,False,2706,429690,WHBG,0,16:10:00,16:14:17,-9.300000,5.016666,15.0,22.850000,7.850000,TO DOWNTOWN,0,1,0


In [13]:
wego['TIME_POINT_ABBR'].head(12)

0         WALM
1         HLWD
2         WHBG
3         CH46
4     28&CHARL
5       MCC5_1
6       MCC5_1
7     28&CHARL
8         CH46
9         WHBG
10        HLWD
11        WALM
Name: TIME_POINT_ABBR, dtype: object

In [38]:
adjusted_counts[(adjusted_counts['ADHERENCE'] < -6)].sort_values('ADHERENCE')

,DATE,DAY_TYPE,CALENDAR_ID,IS_HOLIDAY,OPERATOR,TRIP_ID,TIME_POINT_ABBR,TRIP_EDGE,SCHEDULED_TIME,ACTUAL_ARRIVAL_TIME,ADHERENCE,DWELL_IN_MINS,SCHEDULED_HDWY,ACTUAL_HDWY,HDWY_DEV,ROUTE_DIRECTION_NAME,ADJUSTED_EARLY_COUNT,ADJUSTED_LATE_COUNT,ADJUSTED_ONTIME_COUNT
352298,2024-05-08,Weekday,120240508,False,522,377643,HLWD,0,12:13:00,05:34:00,-725.500000,1124.500000,16.0,12.233333,-3.766667,FROM DOWNTOWN,0,1,0
530746,2025-01-11,Saturday,120250111,False,3402,417746,MCC5_1,1,12:15:00,12:00:18,-561.566666,576.266666,20.0,20.533333,0.533333,FROM DOWNTOWN,0,1,0
162264,2023-08-18,Weekday,120230818,False,2530,354098,MCC5_11,2,15:37:00,21:54:01,-489.316666,112.300000,NaN,NaN,NaN,TO DOWNTOWN,0,1,0
532623,2025-01-13,Weekday,120250113,False,2448,420621,MCC5_11,2,15:37:00,22:35:36,-482.150000,63.550000,NaN,NaN,NaN,TO DOWNTOWN,0,1,0
81271,2023-04-25,Weekday,120230425,False,2497,344404,MCC5_11,2,15:37:00,20:28:00,-481.150000,190.150000,NaN,NaN,NaN,TO DOWNTOWN,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176661,2023-09-08,Weekday,120230908,False,2638,351822,HLWD,0,19:40:00,19:46:01,-6.016666,0.000000,20.0,20.050000,0.050000,FROM DOWNTOWN,0,1,0
321949,2024-03-28,Weekday,120240328,False,1791,371943,28&CHARL,0,16:28:00,16:32:18,-6.016666,1.716666,15.0,14.466666,-0.533334,FROM DOWNTOWN,0,1,0
299281,2024-02-26,Weekday,120240226,False,2012,372043,CH46,0,12:46:00,12:52:01,-6.016666,0.000000,15.0,18.466666,3.466666,FROM DOWNTOWN,0,1,0
328614,2024-04-06,Saturday,120240406,False,3102,377439,HLWD,0,22:08:00,22:13:37,-6.016666,0.400000,30.0,31.950000,1.950000,FROM DOWNTOWN,0,1,0


In [15]:
april28_data = adjusted_counts.loc[
    (adjusted_counts['CALENDAR_ID'] == 120250428)]
    

all_april28_avg_adherence = april28_data['ADHERENCE'].mean()
all_april28_max_adherence = april28_data['ADHERENCE'].max()
all_april28_min_adherence = april28_data['ADHERENCE'].min()

print(f"April 28th, 2025 Adherence")
print(f"Average Adherence: {all_april28_avg_adherence}")
print(f"Maximum Adherence: {all_april28_max_adherence}")
print(f"Minimum Adherence: {all_april28_min_adherence}")

April 28th, 2025 Adherence
Average Adherence: -2.7607095865504356
Maximum Adherence: 21.7
Minimum Adherence: -18.883333


In [16]:
april28_data_whbg = adjusted_counts.loc[
    (adjusted_counts['CALENDAR_ID'] == 120250428) & 
    (adjusted_counts['TIME_POINT_ABBR'] == 'WHBG')]

april28_avg_adherence = april28_data_whbg['ADHERENCE'].mean()
april28_max_adherence = april28_data_whbg['ADHERENCE'].max()
april28_min_adherence = april28_data_whbg['ADHERENCE'].min()

print(f"White Bridge, April 28th, 2025 Adherence")
print(f"Average Adherence: {april28_avg_adherence}")
print(f"Maximum Adherence: {april28_max_adherence}")
print(f"Minimum Adherence: {april28_min_adherence}")

White Bridge, April 28th, 2025 Adherence
Average Adherence: -3.5010098257575755
Maximum Adherence: 1.616666
Minimum Adherence: -18.883333


In [17]:
may5_data_whbg = adjusted_counts.loc[
    (adjusted_counts['CALENDAR_ID'] == 120250505) & 
    (adjusted_counts['TIME_POINT_ABBR'] == 'WHBG')]

may5_avg_adherence = may5_data_whbg['ADHERENCE'].mean()
may5_max_adherence = may5_data_whbg['ADHERENCE'].max()
may5_min_adherence = may5_data_whbg['ADHERENCE'].min()

print(f"White Bridge, May 5th, 2025 Adherence")
print(f"Average Adherence: {may5_avg_adherence}")
print(f"Maximum Adherence: {may5_max_adherence}")
print(f"Minimum Adherence: {may5_min_adherence}")

White Bridge, May 5th, 2025 Adherence
Average Adherence: -3.554615123076923
Maximum Adherence: 1.95
Minimum Adherence: -18.133333


In [18]:
may12_data_whbg = adjusted_counts.loc[
    (adjusted_counts['CALENDAR_ID'] == 120250512) & 
    (adjusted_counts['TIME_POINT_ABBR'] == 'WHBG')]

may12_avg_adherence = may12_data_whbg['ADHERENCE'].mean()
may12_max_adherence = may12_data_whbg['ADHERENCE'].max()
may12_min_adherence = may12_data_whbg['ADHERENCE'].min()

print(f"White Bridge, May 12th, 2025 Adherence")
print(f"Average Adherence: {may12_avg_adherence}")
print(f"Maximum Adherence: {may12_max_adherence}")
print(f"Minimum Adherence: {may12_min_adherence}")

White Bridge, May 12th, 2025 Adherence
Average Adherence: -2.2264628396946566
Maximum Adherence: 1.916666
Minimum Adherence: -12.983333


In [19]:
early_mean = wego['ADJUSTED_EARLY_COUNT'].mean()
late_mean = wego['ADJUSTED_LATE_COUNT'].mean()
ontime_mean = wego['ADJUSTED_ONTIME_COUNT'].mean()

total = early_mean + late_mean + ontime_mean

early_percent = (early_mean / total) * 100
late_percent = (late_mean / total) * 100
ontime_percent = (ontime_mean / total) * 100

print(f"Overall Percentage")
print(f"Early: {early_percent:.2f}%")
print(f"Late: {late_percent:.2f}%")
print(f"Ontime: {ontime_percent:.2f}%")

Overall Percentage
Early: 3.02%
Late: 12.19%
Ontime: 84.80%


The second main variable you'll be looking at is **headway**.  This is the amount of time between a bus and the prior bus at the same stop. In the dataset, the amount of headway scheduled is contained in the SCHEDULED_HDWY column and indicates the difference between the scheduled time for a particular stop and the scheduled time for the previous bus on that same stop.
This dataset contains a column HDWY_DEV, which shows the amount of deviation from the scheduled headway. **Bunching** occurs when there is shorter headway than scheduled, which would appear as a negative HDWY_DEV value. **Gapping** is when there is more headway than scheduled and appears as a positive value in the HDWY_DEV column. Note that you can calculate headway deviation percentage as HDWY_DEV/SCHEDULED_HDWY. The generally accepted range of headway deviation is 50% to 150% of the scheduled headway, so if scheduled headway is 10 minutes, a headway deviation of up to 5 minutes would be acceptable (but not ideal).


In [20]:
non_nan_values = wego['SCHEDULED_HDWY'].dropna()
non_nan_values

12        30.0
13        30.0
14        30.0
15        30.0
16        30.0
          ... 
618983    15.0
618984    15.0
618985    15.0
618986    15.0
618996    10.0
Name: SCHEDULED_HDWY, Length: 503668, dtype: float64

In [39]:
non_null_headway = wego[['DATE', 'DAY_TYPE', 'IS_HOLIDAY', 'SCHEDULED_TIME', 'ACTUAL_ARRIVAL_TIME', 'TIME_POINT_ABBR', 'SCHEDULED_HDWY', 'ACTUAL_HDWY', 'HDWY_DEV']].dropna()
non_null_headway 

,DATE,DAY_TYPE,IS_HOLIDAY,SCHEDULED_TIME,ACTUAL_ARRIVAL_TIME,TIME_POINT_ABBR,SCHEDULED_HDWY,ACTUAL_HDWY,HDWY_DEV
12,2023-01-01,Sunday,False,07:03:00,06:44:01,WALM,30.0,35.883333,5.883333
13,2023-01-01,Sunday,False,07:09:00,07:13:26,HLWD,30.0,33.350000,3.350000
14,2023-01-01,Sunday,False,07:17:00,07:20:48,WHBG,30.0,34.016666,4.016666
15,2023-01-01,Sunday,False,07:20:00,07:24:18,CH46,30.0,34.066666,4.066666
16,2023-01-01,Sunday,False,07:24:00,07:26:41,28&CHARL,30.0,32.516666,2.516666
...,...,...,...,...,...,...,...,...,...
618983,2025-05-12,Weekday,False,17:27:00,17:28:46,28&CHARL,15.0,11.383333,-3.616667
618984,2025-05-12,Weekday,False,17:32:00,17:33:19,CH46,15.0,9.816666,-5.183334
618985,2025-05-12,Weekday,False,17:37:00,17:36:45,WHBG,15.0,10.000000,-5.000000
618986,2025-05-12,Weekday,False,17:46:00,17:46:19,HLWD,15.0,7.500000,-7.500000


In [24]:
non_null_headway['HDWY_DEV_PCT'] = (non_null_headway['HDWY_DEV'] / non_null_headway['SCHEDULED_HDWY']) * 100
non_null_headway.sort_values(['HDWY_DEV'])

,DATE,DAY_TYPE,SCHEDULED_TIME,ACTUAL_ARRIVAL_TIME,TIME_POINT_ABBR,SCHEDULED_HDWY,ACTUAL_HDWY,HDWY_DEV,HDWY_DEV_PCT
100412,2023-05-22,Weekday,15:10:00,14:51:57,MLKS,240.0,6.316666,-233.683334,-97.368056
362784,2024-05-22,Weekday,15:05:00,15:06:23,HEAD,210.0,17.283333,-192.716667,-91.769841
270149,2024-01-17,Saturday,06:32:00,06:36:45,WHBG,63.0,3.966666,-59.033334,-93.703705
530464,2025-01-10,Saturday,05:34:00,05:15:02,WALM,55.0,0.516666,-54.483334,-99.060607
136196,2023-07-13,Weekday,06:15:00,06:03:47,MCC5_1,35.0,0.800000,-34.200000,-97.714286
...,...,...,...,...,...,...,...,...,...
100422,2023-05-23,Weekday,15:10:00,15:01:20,MLKS,0.0,242.433333,242.433333,inf
599041,2025-04-15,Weekday,1900-01-01 01:15:00,1900-01-01 00:57:02,MCC5_1,30.0,358.650000,328.650000,1095.500000
269940,2024-01-16,Weekday,16:15:00,16:15:07,HLWD,15.0,396.016666,381.016666,2540.111107
269942,2024-01-16,Weekday,16:43:00,16:21:25,WALM,19.0,403.283333,384.283333,2022.543858


How has TSP affected these two metrics? Keep in mind that there are many other factors that could also be contributing, so be sure to take into account things like day of the week, time of day, time of year (school in session or not), or other factors that may also be affecting adherence or headway deviation.